# Creation of the Dataframe

In [81]:
#import libraries
!pip install beautifulsoup4
import requests
import bs4 as BeautifulSoup
import pandas as pd

In [82]:
#url of wikipedia
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [83]:
s=requests.Session()
response=s.get(url,timeout=10)
response

<Response [200]>

In [84]:
soup=BeautifulSoup.BeautifulSoup(response.text,'html.parser')
torontotable=soup.find('table',{'class':"wikitable sortable"})

In [85]:
df=pd.read_html(str(torontotable))
# convert list to dataframe
df=pd.DataFrame(df[0])
print(df.head())

  Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront


Clean the data

In [86]:
# Ignore cells with a borough that is Not assigned. Remove rows
df2=df[~df["Borough"].str.contains("Not assigned")==True]

In [87]:
#M5A is already combined in one row with the neighborhoods separated with a comma as shown in row 4  in the below table. (used BautifulSoup)
df2.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [88]:
#If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
#Check For a Substring in the Column "Neighbourhood"
NeighbourhoodNA=df2.loc[df2['Neighbourhood'].str.contains("Not assigned",case=False)]
NeighbourhoodNA.head()

,Postal Code,Borough,Neighbourhood


There are no cells with Not assigned Neighborhood.

In [89]:
df2.shape

(103, 3)

The dataframe has 103 rows and 3 columns

# Get the latitude and the longitude coordinates of each neighborhood

In [50]:
!pip install geocoder
import geocoder # import geocoder

Import the .csv file with coordinates.
To difficult to construct the loop function

In [90]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_04abeae85fc94048947b8ed7bfe090d5 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='glH5Z5O9um1CkoL5IbdSTzORAzFbWCuEGFWpZRoseoST',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_04abeae85fc94048947b8ed7bfe090d5.get_object(Bucket='capstoneproject-donotdelete-pr-egev5iahubzuav',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Create the dataframe

In [92]:
FinalTable=pd.merge(df2,df_data_1, on='Postal Code')
FinalTable.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Explore and cluster the neighborhoods in Toronto